In [1]:
def P(var, evidence={}):
    row = tuple(evidence[parent] for parent in var.parents)
    return var.cpt[row]


def normalize(dist):
    total = sum(dist.values())
    for key in dist:
        dist[key] = dist[key] / total
        assert 0 <= dist[key] <= 1, "Probabilities must be between 0 and 1."
    return dist


def sample(probdist):
    r = random.random()
    c = 0.0
    for outcome in probdist:
        c += probdist[outcome]
        if r <= c:
            return outcome


def globalize(mapping):
    globals().update(mapping)

In [2]:
from collections import defaultdict, Counter
import itertools
import math
import random


class BayesNet(object):
    def __init__(self):
        self.variables = []
        self.lookup = {}

    def add(self, name, parentnames, cpt):
        parents = [self.lookup[name] for name in parentnames]
        var = Variable(name, cpt, parents)
        self.variables.append(var)
        self.lookup[name] = var
        return self


class Variable(object):
    def __init__(self, name, cpt, parents=()):
        self.__name__ = name
        self.parents = parents
        self.cpt = CPTable(cpt, parents)
        self.domain = set(itertools.chain(*self.cpt.values()))

    def __repr__(self):
        return self.__name__


class Factor(dict):
    "An {outcome: frequency} mapping."


class ProbDist(Factor):
    def __init__(self, mapping=(), **kwargs):
        if isinstance(mapping, float):
            mapping = {T: mapping, F: 1 - mapping}
        self.update(mapping, **kwargs)
        normalize(self)


class Evidence(dict):
    "A {variable: value} mapping, describing what we know for sure."


class CPTable(dict):
    def __init__(self, mapping, parents=()):
        if len(parents) == 0 and not (
            isinstance(mapping, dict) and set(mapping.keys()) == {()}
        ):
            mapping = {(): mapping}
        for row, dist in mapping.items():
            if len(parents) == 1 and not isinstance(row, tuple):
                row = (row,)
            self[row] = ProbDist(dist)


class Bool(int):
    __str__ = __repr__ = lambda self: "T" if self else "F"


T = Bool(True)
F = Bool(False)


In [3]:
Earthquake = Variable("Earthquake", 0.002)


In [4]:
P(Earthquake)

{T: 0.002, F: 0.998}

In [5]:
Counter(sample(P(Earthquake)) for i in range(100000))

Counter({F: 99790, T: 210})

In [6]:
alarm_net = (
    BayesNet()
    .add("Burglary", [], 0.001)
    .add("Earthquake", [], 0.002)
    .add(
        "Alarm",
        ["Burglary", "Earthquake"],
        {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001},
    )
    .add("JohnCalls", ["Alarm"], {T: 0.90, F: 0.05})
    .add("MaryCalls", ["Alarm"], {T: 0.70, F: 0.01})
)
globalize(alarm_net.lookup)
alarm_net.variables

[Burglary, Earthquake, Alarm, JohnCalls, MaryCalls]

In [7]:
P(Burglary)

{T: 0.001, F: 0.999}

In [8]:
P(Alarm, {Burglary: T, Earthquake: F})

{T: 0.94, F: 0.06000000000000005}

In [9]:
Alarm.cpt

{(T, T): {T: 0.95, F: 0.050000000000000044},
 (T, F): {T: 0.94, F: 0.06000000000000005},
 (F, T): {T: 0.29, F: 0.71},
 (F, F): {T: 0.001, F: 0.999}}

In [10]:
def joint_distribution(net):
    return ProbDist(
        {
            row: prod(P_xi_given_parents(var, row, net) for var in net.variables)
            for row in all_rows(net)
        }
    )


def all_rows(net):
    return itertools.product(*[var.domain for var in net.variables])


def P_xi_given_parents(var, row, net):
    dist = P(var, Evidence(zip(net.variables, row)))
    xi = row[net.variables.index(var)]
    return dist[xi]


def prod(numbers):
    result = 1
    for x in numbers:
        result *= x
    return result


In [11]:
set(all_rows(alarm_net))

{(F, F, F, F, F),
 (F, F, F, F, T),
 (F, F, F, T, F),
 (F, F, F, T, T),
 (F, F, T, F, F),
 (F, F, T, F, T),
 (F, F, T, T, F),
 (F, F, T, T, T),
 (F, T, F, F, F),
 (F, T, F, F, T),
 (F, T, F, T, F),
 (F, T, F, T, T),
 (F, T, T, F, F),
 (F, T, T, F, T),
 (F, T, T, T, F),
 (F, T, T, T, T),
 (T, F, F, F, F),
 (T, F, F, F, T),
 (T, F, F, T, F),
 (T, F, F, T, T),
 (T, F, T, F, F),
 (T, F, T, F, T),
 (T, F, T, T, F),
 (T, F, T, T, T),
 (T, T, F, F, F),
 (T, T, F, F, T),
 (T, T, F, T, F),
 (T, T, F, T, T),
 (T, T, T, F, F),
 (T, T, T, F, T),
 (T, T, T, T, F),
 (T, T, T, T, T)}

In [12]:
row = (F, F, F, F, F)
P(Alarm, {Burglary: F, Earthquake: F})

{T: 0.001, F: 0.999}

In [13]:
P_xi_given_parents(Alarm, row, alarm_net)

0.999

In [14]:
joint_distribution(alarm_net)

{(F, F, F, F, F): 0.9367427006190001,
 (F, F, F, F, T): 0.009462047481000001,
 (F, F, F, T, F): 0.04930224740100002,
 (F, F, F, T, T): 0.0004980024990000002,
 (F, F, T, F, F): 2.9910060000000004e-05,
 (F, F, T, F, T): 6.979013999999999e-05,
 (F, F, T, T, F): 0.00026919054000000005,
 (F, F, T, T, T): 0.00062811126,
 (F, T, F, F, F): 0.0013341744900000002,
 (F, T, F, F, T): 1.3476510000000005e-05,
 (F, T, F, T, F): 7.021971000000001e-05,
 (F, T, F, T, T): 7.092900000000001e-07,
 (F, T, T, F, F): 1.7382600000000002e-05,
 (F, T, T, F, T): 4.0559399999999997e-05,
 (F, T, T, T, F): 0.00015644340000000006,
 (F, T, T, T, T): 0.00036503460000000007,
 (T, F, F, F, F): 5.631714000000006e-05,
 (T, F, F, F, T): 5.688600000000006e-07,
 (T, F, F, T, F): 2.9640600000000033e-06,
 (T, F, F, T, T): 2.9940000000000035e-08,
 (T, F, T, F, F): 2.8143600000000003e-05,
 (T, F, T, F, T): 6.56684e-05,
 (T, F, T, T, F): 0.0002532924000000001,
 (T, F, T, T, T): 0.0005910156000000001,
 (T, T, F, F, F): 9.4050000000

In [15]:
alarm_net2 = (
    BayesNet()
    .add("Hungry", [], 0.05)
    .add("FoodAvailable", [], 0.1)
    .add(
        "Eat",
        ["Hungry", "FoodAvailable"],
        {(T, T): 0.95, (T, F): 0.0, (F, T): 0.05, (F, F): 0.0},
    )
    .add("Sated", ["Eat"], {T: 0.9, F: 0.1})
    .add("Unsated", ["Eat"], {T: 0.7, F: 0.1})
)
globalize(alarm_net2.lookup)
alarm_net2.variables


[Hungry, FoodAvailable, Eat, Sated, Unsated]

In [16]:
Eat.cpt

{(T, T): {T: 0.95, F: 0.050000000000000044},
 (T, F): {T: 0.0, F: 1.0},
 (F, T): {T: 0.05, F: 0.95},
 (F, F): {T: 0.0, F: 1.0}}

In [17]:
P(Hungry)

{T: 0.05, F: 0.95}

In [18]:
P(Eat, {Hungry: T, FoodAvailable: F})

{T: 0.0, F: 1.0}

In [19]:
row = (F, T, F, F, F)
P_xi_given_parents(Eat, row, alarm_net2)

0.95

In [20]:
joint_distribution(alarm_net2)

{(F, F, F, F, F): 0.6925500000000001,
 (F, F, F, F, T): 0.07695000000000002,
 (F, F, F, T, F): 0.07695000000000002,
 (F, F, F, T, T): 0.008550000000000002,
 (F, F, T, F, F): 0.0,
 (F, F, T, F, T): 0.0,
 (F, F, T, T, F): 0.0,
 (F, F, T, T, T): 0.0,
 (F, T, F, F, F): 0.07310250000000001,
 (F, T, F, F, T): 0.008122500000000003,
 (F, T, F, T, F): 0.008122500000000001,
 (F, T, F, T, T): 0.0009025000000000003,
 (F, T, T, F, F): 0.00014250000000000002,
 (F, T, T, F, T): 0.0003325,
 (F, T, T, T, F): 0.0012825000000000007,
 (F, T, T, T, T): 0.0029925000000000012,
 (T, F, F, F, F): 0.03645000000000002,
 (T, F, F, F, T): 0.0040500000000000015,
 (T, F, F, T, F): 0.0040500000000000015,
 (T, F, F, T, T): 0.0004500000000000002,
 (T, F, T, F, F): 0.0,
 (T, F, T, F, T): 0.0,
 (T, F, T, T, F): 0.0,
 (T, F, T, T, T): 0.0,
 (T, T, F, F, F): 0.0002025000000000003,
 (T, T, F, F, T): 2.2500000000000032e-05,
 (T, T, F, T, F): 2.250000000000003e-05,
 (T, T, F, T, T): 2.5000000000000036e-06,
 (T, T, T, F, F): 0

In [21]:
alarm_net3=(
    BayesNet()
    .add("Cloudy", [], 0.5)
    .add("Sprinkler", ["Cloudy"], {(T):0.1,(F):0.5})
    .add("Rain", ["Cloudy"], {(T):0.8,(F):0.2})
    .add(
        "WetGrass",
        ["Sprinkler", "Rain"],
        {(T, T): 0.99, (T, F): 0.9, (F, T): 0.9, (F, F): 0.0},
    )
)
globalize(alarm_net3.lookup)
alarm_net3.variables


[Cloudy, Sprinkler, Rain, WetGrass]

In [22]:
WetGrass.cpt

{(T, T): {T: 0.99, F: 0.010000000000000009},
 (T, F): {T: 0.9, F: 0.09999999999999998},
 (F, T): {T: 0.9, F: 0.09999999999999998},
 (F, F): {T: 0.0, F: 1.0}}

In [23]:
row = (F, T, F, F)
print(P_xi_given_parents(WetGrass, row, alarm_net3))

0.09999999999999998


In [24]:
joint_distribution(alarm_net3)

{(F, F, F, F): 0.19999999999999996,
 (F, F, F, T): 0.0,
 (F, F, T, F): 0.004999999999999998,
 (F, F, T, T): 0.045,
 (F, T, F, F): 0.019999999999999993,
 (F, T, F, T): 0.18,
 (F, T, T, F): 0.0005000000000000003,
 (F, T, T, T): 0.04949999999999999,
 (T, F, F, F): 0.08999999999999997,
 (T, F, F, T): 0.0,
 (T, F, T, F): 0.03599999999999999,
 (T, F, T, T): 0.324,
 (T, T, F, F): 0.0009999999999999994,
 (T, T, F, T): 0.008999999999999998,
 (T, T, T, F): 0.00040000000000000034,
 (T, T, T, T): 0.0396}

In [25]:
print("P(S | R, W, C):",P_xi_given_parents(Sprinkler, (T,T,T,T), alarm_net3))

P(S | R, W, C): 0.1


In [26]:
print("P(S | ~C, ~W, R):",P_xi_given_parents(Sprinkler, (F,T,T,F), alarm_net3))

P(S | ~C, ~W, R): 0.5


# Monty Hall

In [27]:
import random


def run_trial(switch_doors, ndoors=3):
    chosen_door = random.randint(1, ndoors)
    if switch_doors:
        revealed_door = 3 if chosen_door == 2 else 2
        available_doors = [
            dnum
            for dnum in range(1, ndoors + 1)
            if dnum not in (chosen_door, revealed_door)
        ]
        chosen_door = random.choice(available_doors)

    return chosen_door == 1


def run_trials(ntrials, switch_doors, ndoors=3):
    nwins = 0
    for i in range(ntrials):
        if run_trial(switch_doors, ndoors):
            nwins += 1
    return nwins


ndoors, ntrials = 3, 10000
nwins_without_switch = run_trials(ntrials, False, ndoors)
nwins_with_switch = run_trials(ntrials, True, ndoors)


print(f"Monty Hall Problem with {ndoors} doors")
print(f"Proportion of wins without switching: {nwins_without_switch/ntrials:.4f}")
print(f"Proportion of wins with switching: {nwins_with_switch / ntrials:.4f}")


Monty Hall Problem with 3 doors
Proportion of wins without switching: 0.3342
Proportion of wins with switching: 0.6675
